In [1]:
using DataFrames
using CSV
using TextAnalysis
using MLJ
using Chain
using Pipe
using StableRNGs
using StringEncodings
using MLJText
using MLJBase
using Plots

# Create Dataframe From CSV

In [4]:
df = CSV.File(open("training.1600000.processed.noemoticon.csv", enc"ISO-8859-1"); header=0) |> DataFrame
rename!(df, [:Target, :Id, :Date, :Flag, :User, :Tweet])

df = select!(df, Not([:Id, :Date, :Flag, :User]))

df

Row,Target,Tweet
,Int64,String
1,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
2,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
3,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
4,0,my whole body feels itchy and like its on fire
5,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
6,0,@Kwesidei not the whole crew
7,0,Need a hug
8,0,"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?"
9,0,@Tatiana_K nope they didn't have it


# Cast Type

In [5]:
df.Target = coerce(df.Target, OrderedFactor)
levels(df.Target) 

df.Tweet = TextAnalysis.StringDocument.(df[:, :Tweet])

1600000-element Vector{StringDocument{String}}:
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 ⋮
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}
 A StringDocument{String}

# Extract Tweet and Target As Separate Variables

In [6]:
feat, target = MLJ.unpack(df, ==(:Tweet), ==(:Target))

(StringDocument{String}[A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}  …  A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}, A StringDocument{String}], CategoricalArrays.CategoricalValue{Int64, UInt32}[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  4, 4, 4, 4, 4, 4, 4, 4, 4, 4], 0×0 DataFrame)

In [8]:
prepare!.(feat, strip_case| strip_pronouns| strip_articles| strip_punctuation| strip_numbers| strip_non_letters| strip_pronouns| strip_stopwords| stem_words)

crps = Corpus(feat)

update_lexicon!(crps)

rng = StableRNG(100)
train, test = partition(eachindex(target), 0.7, shuffle=true, rng=rng);

In [9]:
MultinomialNBClassifier = @load MultinomialNBClassifier pkg=NaiveBayes
multinomial_nb_classifier_pipe = (feat -> tokenize.(TextAnalysis.text.(feat))) |> TfidfTransformer() |> (feat -> round.(feat)) |> MultinomialNBClassifier()

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/bryan/.julia/packages/MLJModels/OJDDo/src/loading.jl:159


import MLJNaiveBayesInterface ✔

ProbabilisticPipeline(
  f = var"#9#11"(), 
  tfidf_transformer = TfidfTransformer(
        max_doc_freq = 1.0, 
        min_doc_freq = 0.0, 
        smooth_idf = true), 
  f2 = var"#10#12"(), 
  multinomial_nb_classifier = MultinomialNBClassifier(
        alpha = 1), 
  cache = true)

In [12]:
mach = machine(multinomial_nb_classifier_pipe, feat, target)
MLJ.fit!(mach, rows=train)

┌ Info: Training machine(ProbabilisticPipeline(f = #9, …), …).
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:492


┌ Info: Training machine(:tfidf_transformer, …).
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:492


┌ Info: Training machine(:multinomial_nb_classifier, …).
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:492
┌ Error: Problem fitting the machine machine(:multinomial_nb_classifier, …). 
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:682


┌ Info: Running type checks... 
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:688
┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc NaiveBayes.MultinomialNBClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{AbstractMatrix{Continuous}, AbstractVector{OrderedFactor{2}}}
│ 
│ fit_data_scitype(model) = Tuple{Union{Table{<:AbstractVector{<:Count}}, AbstractMatrix{<:Count}}, A

┌ Info: It seems an upstream node in a learning network is providing data of incompatible scitype. See above. 
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:694
┌ Error: Problem fitting machine(:multinomial_nb_classifier, …)
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:755
┌ Error: Problem fitting the machine machine(ProbabilisticPipeline(f = #9, …), …). 
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:682
┌ Info: Running type checks... 
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:688


┌ Info: Type checks okay. 
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/WKVEo/src/machines.jl:692


CompositeException: TaskFailedException

    nested task error: OutOfMemoryError()
    Stacktrace:
     [1] fit_only!(mach::Machine{Symbol, true}, wait_on_upstream::Bool; kwargs::Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol}, NamedTuple{(:verbosity, :composite), Tuple{Int64, ProbabilisticPipeline{NamedTuple{(:f, :tfidf_transformer, :f2, :multinomial_nb_classifier), Tuple{Any, Unsupervised, Any, Probabilistic}}, MLJModelInterface.predict}}}})
       @ MLJBase ~/.julia/packages/MLJBase/WKVEo/src/machines.jl:756
     [2] (::MLJBase.var"#80#82"{Base.Pairs{Symbol, Any, Tuple{Symbol, Symbol}, NamedTuple{(:verbosity, :composite), Tuple{Int64, ProbabilisticPipeline{NamedTuple{(:f, :tfidf_transformer, :f2, :multinomial_nb_classifier), Tuple{Any, Unsupervised, Any, Probabilistic}}, MLJModelInterface.predict}}}}, Machine{Symbol, true}})()
       @ MLJBase ./task.jl:484
    
    caused by: OutOfMemoryError()
    Stacktrace:
     [1] Array
       @ ./boot.jl:461 [inlined]
     [2] Array
       @ ./

In [ ]:
evaluation = evaluate!(mach, resampling=CV(), measure=[Accuracy(), Precision(), TruePositiveRate(), FScore(), LogLoss()], rows=test)
println(evaluation)

In [ ]:
yhat = MLJ.predict(mach, rows=test)

println("Accuracy on the test set: $(MLJ.accuracy(mode.(yhat), target[test]))")
println("fscore on the test set: $(MLJ.f1score(mode.(yhat), target[test]))")

confusion_mat = ConfusionMatrix()(mode.(yhat), coerce(target[test], OrderedFactor))
cm_plt = Plots.heatmap(confusion_mat.mat, xlabel="ground truth", ylabel="predicted values", title="Confusion Matrix of\nMNB Classifier using Rounded TF-IDF Vectorizer", size= (800, 800))